In [ ]:
def prepare_features(df, args, add_year=False):
    global training_features
    features = copy.deepcopy(training_features)
    df['dimuon_pt_log'] = np.log(df['dimuon_pt'])
    df['jj_mass_log'] = np.log(df['jj_mass'])
    if add_year and ('year' not in features):
        features+=['year']
    if not add_year and ('year' in features):
        features = [t for t in features if t!='year']
    for trf in features:
        if trf not in df.columns:
            print(f'Variable {trf} not found in training dataframe!')
    return df, features
    
def dnn_training(df, args, model):
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Dense, Activation, Input, Dropout, Concatenate, Lambda, BatchNormalization
    from tensorflow.keras import backend as K

    def scale_data(inputs, model, label):
        x_mean = np.mean(x_train[inputs].values,axis=0)
        x_std = np.std(x_train[inputs].values,axis=0)
        training_data = (x_train[inputs]-x_mean)/x_std
        validation_data = (x_val[inputs]-x_mean)/x_std
        np.save(f"output/trained_models/{model}/scalers_{label}", [x_mean, x_std])
        return training_data, validation_data

    nfolds = 4
    classes = df.cls.unique()
    cls_idx_map = {cls:idx for idx,cls in enumerate(classes)}
    add_year = (args['year']=='')
    df, features = prepare_features(df, args, add_year)
    df['cls_idx'] = df['cls'].map(cls_idx_map)
    
    try:
        os.mkdir(f"output/trained_models/{model}/")
    except:
        pass
    
    print("Training features: ", features)
    for i in range(nfolds):
        if args['year']=='':
            label = f"allyears_{args['label']}_{i}"
        else:
            label = f"allyears_{args['year']}_{args['label']}_{i}"
        
        train_folds = [(i+f)%nfolds for f in [0,1]]
        val_folds = [(i+f)%nfolds for f in [2]]
        eval_folds = [(i+f)%nfolds for f in [3]]

        print(f"Train classifier #{i+1} out of {nfolds}")
        print(f"Training folds: {train_folds}")
        print(f"Validation folds: {val_folds}")
        print(f"Evaluation folds: {eval_folds}")
        print(f"Samples used: ",df.s.unique())

        train_filter = df.event.mod(nfolds).isin(train_folds)
        val_filter = df.event.mod(nfolds).isin(val_folds)
        eval_filter = df.event.mod(nfolds).isin(eval_folds)
        
        other_columns = ['event', 'wgt_nominal']
        
        df_train = df[train_filter]
        df_val = df[val_filter]
        
        x_train = df_train[features]
        y_train = df_train['cls_idx']
        x_val = df_val[features]
        y_val = df_val['cls_idx']

        df_train['cls_avg_wgt'] = 1.0
        df_val['cls_avg_wgt'] = 1.0
        for icls, cls in enumerate(classes):
            train_evts = len(y_train[y_train==icls])
            df_train.loc[y_train==icls,'cls_avg_wgt'] = df_train.loc[y_train==icls,'wgt_nominal'].values.mean()
            df_val.loc[y_val==icls,'cls_avg_wgt'] = df_val.loc[y_val==icls,'wgt_nominal'].values.mean()
            print(f"{train_evts} training events in class {cls}")

        for smp in df_train.s.unique():
            df_train.loc[df_train.s==smp,'smp_avg_wgt'] = df_train.loc[df_train.s==smp,'wgt_nominal'].values.mean()
            df_val.loc[df_val.s==smp,'smp_avg_wgt'] = df_val.loc[df_val.s==smp,'wgt_nominal'].values.mean()
            print(f"{train_evts} training events in class {cls}")        
            
#        df_train['training_wgt'] = df_train['wgt_nominal']/df_train['cls_avg_wgt']
#        df_val['training_wgt'] = df_val['wgt_nominal']/df_val['cls_avg_wgt']
        df_train['training_wgt'] = df_train['wgt_nominal']/df_train['smp_avg_wgt']
        df_val['training_wgt'] = df_val['wgt_nominal']/df_val['smp_avg_wgt']

        
        # scale data
        x_train, x_val = scale_data(features, model, label)
        x_train[other_columns] = df_train[other_columns]
        x_val[other_columns] = df_val[other_columns]

        # load model
        input_dim = len(features)
        inputs = Input(shape=(input_dim,), name = label+'_input')
        x = Dense(128, name = label+'_layer_1', activation='tanh')(inputs)
        x = Dropout(0.2)(x)
        x = BatchNormalization()(x)
        x = Dense(64, name = label+'_layer_2', activation='tanh')(x)
        x = Dropout(0.2)(x)
        x = BatchNormalization()(x)
        x = Dense(32, name = label+'_layer_3', activation='tanh')(x)
        x = Dropout(0.2)(x)
        x = BatchNormalization()(x)
#        x = Dense(8, name = label+'_layer_4', activation='tanh')(x)
#        x = Dropout(0.2)(x)
#        x = BatchNormalization()(x)
#        x = Dense(8, name = label+'_layer_5', activation='tanh')(x)
#        x = Dropout(0.2)(x)
#        x = BatchNormalization()(x)def prepare_features(df, args, add_year=False):
    global training_features
    features = copy.deepcopy(training_features)
    df['dimuon_pt_log'] = np.log(df['dimuon_pt'])
    df['jj_mass_log'] = np.log(df['jj_mass'])
    if add_year and ('year' not in features):
        features+=['year']
    if not add_year and ('year' in features):
        features = [t for t in features if t!='year']
    for trf in features:
        if trf not in df.columns:
            print(f'Variable {trf} not found in training dataframe!')
    return df, features
    
      

def evaluation(df, args):
    if df.shape[0]==0: return df
    for model in args['dnn_models']:
        df = dnn_evaluation(df, model, args)
    for model in args['bdt_models']:
        df = bdt_evaluation(df, model, args)
    return df

        outputs = Dense(1, name = label+'_output',  activation='sigmoid')(x)

        dnn = Model(inputs=inputs, outputs=outputs)
        dnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
        dnn.summary()

#        history = dnn.fit(x_train[features], y_train, epochs=100, batch_size=1024,\
#                            sample_weight=df_train['training_wgt'].values, verbose=1,\
#                            validation_data=(x_val[features], y_val, df_val['training_wgt'].values), shuffle=True)
        history = dnn.fit(x_train[features], y_train, epochs=100, batch_size=1024, verbose=1,\
                            validation_data=(x_val[features], y_val), shuffle=True)

        util.save(history.history, f"output/trained_models/{model}/history_{label}.coffea")
        dnn.save(f"output/trained_models/{model}/dnn_{label}.h5")        

def evaluation(df, args):
    if df.shape[0]==0: return df
    for model in args['dnn_models']:
        df = dnn_evaluation(df, model, args)
    for model in args['bdt_models']:
        df = bdt_evaluation(df, model, args)
    return df


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, n_feat):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_feat, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.dropout3 = nn.Dropout(0.2)
        self.output = nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.tanh(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = F.tanh(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = F.tanh(x)
        x = self.dropout3(x)

        x = self.output(x)
        output = F.sigmoid(x)
        return output

def scale_data(inputs, model, label):
    x_mean = np.mean(x_train[inputs].values,axis=0)
    x_std = np.std(x_train[inputs].values,axis=0)
    training_data = (x_train[inputs]-x_mean)/x_std
    validation_data = (x_val[inputs]-x_mean)/x_std
    np.save(f"output/trained_models/{model}/scalers_{label}", [x_mean, x_std])
    return training_data, validation_data

training_features = [
    'dimuon_mass', 'dimuon_pt', 'dimuon_pt_log', 'dimuon_eta', 'dimuon_mass_res', 'dimuon_mass_res_rel',\
     'dimuon_cos_theta_cs', 'dimuon_phi_cs',
     'jet1_pt', 'jet1_eta', 'jet1_phi', 'jet1_qgl', 'jet2_pt', 'jet2_eta', 'jet2_phi', 'jet2_qgl',\
     'jj_mass', 'jj_mass_log', 'jj_dEta', 'rpt', 'll_zstar_log', 'mmj_min_dEta', 'nsoftjets5', 'htsoft2'
                    ]
# TODO: input year as features and other stuff
def dnn_training(df, args, model):
    nfolds = 4
    classes = df.cls.unique()
    cls_idx_map = {cls:idx for idx,cls in enumerate(classes)}
    add_year = (args['year']=='')
    df, features = prepare_features(df, args, add_year)
    df['cls_idx'] = df['cls'].map(cls_idx_map)
    
    try:
        os.mkdir(f"output/trained_models/{model}/")
    except:
        pass
    
    print("Training features: ", features)
    for i in range(nfolds):
        if args['year']=='':
            label = f"allyears_{args['label']}_{i}"
        else:
            label = f"allyears_{args['year']}_{args['label']}_{i}"
        
        train_folds = [(i+f)%nfolds for f in [0,1]]
        val_folds = [(i+f)%nfolds for f in [2]]
        eval_folds = [(i+f)%nfolds for f in [3]]

        print(f"Train classifier #{i+1} out of {nfolds}")
        print(f"Training folds: {train_folds}")
        print(f"Validation folds: {val_folds}")
        print(f"Evaluation folds: {eval_folds}")
        print(f"Samples used: ",df.s.unique())

        train_filter = df.event.mod(nfolds).isin(train_folds)
        val_filter = df.event.mod(nfolds).isin(val_folds)
        eval_filter = df.event.mod(nfolds).isin(eval_folds)
        
        other_columns = ['event', 'wgt_nominal']
        
        df_train = df[train_filter]
        df_val = df[val_filter]
        
        x_train = df_train[features]
        y_train = df_train['cls_idx']
        x_val = df_val[features]
        y_val = df_val['cls_idx']

        df_train['cls_avg_wgt'] = 1.0
        df_val['cls_avg_wgt'] = 1.0
        for icls, cls in enumerate(classes):
            train_evts = len(y_train[y_train==icls])
            df_train.loc[y_train==icls,'cls_avg_wgt'] = df_train.loc[y_train==icls,'wgt_nominal'].values.mean()
            df_val.loc[y_val==icls,'cls_avg_wgt'] = df_val.loc[y_val==icls,'wgt_nominal'].values.mean()
            print(f"{train_evts} training events in class {cls}")

        for smp in df_train.s.unique():
            df_train.loc[df_train.s==smp,'smp_avg_wgt'] = df_train.loc[df_train.s==smp,'wgt_nominal'].values.mean()
            df_val.loc[df_val.s==smp,'smp_avg_wgt'] = df_val.loc[df_val.s==smp,'wgt_nominal'].values.mean()
            print(f"{train_evts} training events in class {cls}")        
            
#        df_train['training_wgt'] = df_train['wgt_nominal']/df_train['cls_avg_wgt']
#        df_val['training_wgt'] = df_val['wgt_nominal']/df_val['cls_avg_wgt']
        df_train['training_wgt'] = df_train['wgt_nominal']/df_train['smp_avg_wgt']
        df_val['training_wgt'] = df_val['wgt_nominal']/df_val['smp_avg_wgt']

        
        # scale data
        x_train, x_val = scale_data(features, model, label)
        x_train[other_columns] = df_train[other_columns]
        x_val[other_columns] = df_val[other_columns]

        # load model
        input_dim = len(features)
        inputs = Input(shape=(input_dim,), name = label+'_input')
        x = Dense(128, name = label+'_layer_1', activation='tanh')(inputs)
        x = Dropout(0.2)(x)
        x = BatchNormalization()(x)
        x = Dense(64, name = label+'_layer_2', activation='tanh')(x)
        x = Dropout(0.2)(x)
        x = BatchNormalization()(x)
        x = Dense(32, name = label+'_layer_3', activation='tanh')(x)
        x = Dropout(0.2)(x)
        x = BatchNormalization()(x)
#        x = Dense(8, name = label+'_layer_4', activation='tanh')(x)
#        x = Dropout(0.2)(x)
#        x = BatchNormalization()(x)
#        x = Dense(8, name = label+'_layer_5', activation='tanh')(x)
#        x = Dropout(0.2)(x)
#        x = BatchNormalization()(x)
        outputs = Dense(1, name = label+'_output',  activation='sigmoid')(x)

        dnn = Model(inputs=inputs, outputs=outputs)
        dnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
        dnn.summary()

#        history = dnn.fit(x_train[features], y_train, epochs=100, batch_size=1024,\
#                            sample_weight=df_train['training_wgt'].values, verbose=1,\
#                            validation_data=(x_val[features], y_val, df_val['training_wgt'].values), shuffle=True)
        history = dnn.fit(x_train[features], y_train, epochs=100, batch_size=1024, verbose=1,\
                            validation_data=(x_val[features], y_val), shuffle=True)

        util.save(history.history, f"output/trained_models/{model}/history_{label}.coffea")
        dnn.save(f"output/trained_models/{model}/dnn_{label}.h5")  

In [18]:
import dask_awkward as dak
import numpy as np
import awkward as ak
import glob

base_path = f"/depot/cms/users/yun79/hmm/copperheadV1clean/V2_Dec22_HEMVetoOnZptOn_RerecoBtagSF_XS_Rereco/stage1_output/2018/f1_0/"

# def getParquetFiles(path):
    # return glob.glob(path)

def fillEventNans(events):
    for field in events.fields:
        if "phi" in field:
            events[field] = ak.fill_none(events[field], value=-10) # we're working on a DNN, so significant deviation may be warranted
        else: # for all other fields (this may need to be changed)
            events[field] = ak.fill_none(events[field], value=0)
    return events

# def replaceSidebandMass(events):
#     for field in events.fields:
#         if "phi" in field:
#             events[field] = ak.fill_none(events[field], value=-1)
#         else: # for all other fields (this may need to be changed)
#             events[field] = ak.fill_none(events[field], value=0)
#     return events

def applyCatAndFeatFilter(events, features: list, region="h-peak", category="vbf"):
    """
    
    """
    # apply category filter
    dimuon_mass = events.dimuon_mass
    if region =="h-peak":
        region = (dimuon_mass > 115.03) & (dimuon_mass < 135.03)
    elif region =="h-sidebands":
        region = ((dimuon_mass > 110) & (dimuon_mass < 115.03)) | ((dimuon_mass > 135.03) & (dimuon_mass < 150))
    elif region =="signal":
        region = (dimuon_mass >= 110) & (dimuon_mass <= 150.0)
    
    if category.lower() == "vbf":
        btag_cut =ak.fill_none((events.nBtagLoose_nominal >= 2), value=False) | ak.fill_none((events.nBtagMedium_nominal >= 1), value=False)
        cat_cut = (events.jj_mass_nominal > 400) & (events.jj_dEta_nominal > 2.5) & (events.jet1_pt_nominal > 35) 
        cat_cut = cat_cut & (~btag_cut) # btag cut is for VH and ttH categories
    elif category.lower()== "ggh":
        btag_cut =ak.fill_none((events.nBtagLoose_nominal >= 2), value=False) | ak.fill_none((events.nBtagMedium_nominal >= 1), value=False)
        cat_cut = (events.jj_mass_nominal > 400) & (events.jj_dEta_nominal > 2.5)
        cat_cut = cat_cut & (~btag_cut) # btag cut is for VH and ttH categories
    else: # no category cut is applied
        cat_cut = ak.ones_like(dimuon_mass, dtype="bool")
    cat_cut = ak.fill_none(cat_cut, value=False)
    cat_filter = (
        cat_cut & 
        region 
    )
    events = events[cat_filter] # apply the category filter
    print(f"events dimuon_mass: {events.dimuon_mass.compute()}")
    # apply the feature filter (so the ak zip only contains features we are interested)
    print(f"features: {features}")
    events = ak.zip({field : events[field] for field in features}) 
    return events


def prepare_features(events, features, variation="nominal"):
    features_var = []
    for trf in features:
        if "soft" in trf:
            variation_current = "nominal"
        else:
            variation_current = variation
        
        if f"{trf}_{variation_current}" in events.fields:
            features_var.append(f"{trf}_{variation_current}")
        elif trf in events.fields:
            features_var.append(trf)
        else:
            print(f"Variable {trf} not found in training dataframe!")
    return features_var

def preprocess(base_path, region="h-peak", category="vbf"):
    # training_features = [
    #     "dimuon_mass",
    #     "dimuon_pt",
    #     "dimuon_pt_log",
    #     "dimuon_eta",
    #     # "dimuon_ebe_mass_res",
    #     # "dimuon_ebe_mass_res_rel",
    #     # "dimuon_cos_theta_cs",
    #     # "dimuon_phi_cs",
    #     "dimuon_pisa_mass_res",
    #     "dimuon_pisa_mass_res_rel",
    #     "dimuon_cos_theta_cs_pisa",
    #     "dimuon_phi_cs_pisa",
    #     "jet1_pt",
    #     "jet1_eta",
    #     "jet1_phi",
    #     "jet1_qgl",
    #     "jet2_pt",
    #     "jet2_eta",
    #     "jet2_phi",
    #     "jet2_qgl",
    #     "jj_mass",
    #     "jj_mass_log",
    #     "jj_dEta",
    #     "rpt",
    #     "ll_zstar_log",
    #     "mmj_min_dEta",
    #     "nsoftjets5",
        # "htsoft2",
        # "year",
    # ]
    training_features = [
        'dimuon_mass', 'dimuon_pt', 'dimuon_pt_log', 'dimuon_eta', \
         'dimuon_cos_theta_cs', 'dimuon_phi_cs',
         'jet1_pt', 'jet1_eta', 'jet1_phi', 'jet1_qgl', 'jet2_pt', 'jet2_eta', 'jet2_phi', 'jet2_qgl',\
         'jj_mass', 'jj_mass_log', 'jj_dEta', 'rpt', 'll_zstar_log', 'mmj_min_dEta', 'nsoftjets5', 'htsoft2'
    ]
    # TODO: add mixup
    # sig and bkg processes defined at line 1976 of AN-19-124. IDK why ggH is not included here
    sig_processes = ["vbf_powheg_dipole"]
    bkg_processes = ["dy_M-100To200", "ewk_lljj_mll105_160_ptj0","ttjets_dl","ttjets_sl"]
    
    filenames = []
    for process in sig_processes:
        filenames += glob.glob(f"{base_path}/{process}/*/*.parquet")
    # print(filenames)
    events = dak.from_parquet(filenames)
    training_features = prepare_features(events, training_features) # add variation to features
    events = applyCatAndFeatFilter(events, training_features, region=region, category=category)
    events = fillEventNans(events)
    df = ak.to_dataframe(events.compute())
    print(f"df: {df.head()}")
    # turn to pandas df add label (signal=1, bkg=0)
    # merge sig and bkg dfs
    # calculate the scale, save it
    # save the resulting df for training
    
    
    filenames = []
    for process in bkg_processes:
        filenames += glob.glob(f"{base_path}/{process}/*/*.parquet")
    # print(filenames)
    bkg_events = dak.from_parquet(filenames)

    # fillnan values
preprocess(base_path)

events dimuon_mass: [126, 125, 122, 124, 125, 128, 121, 129, ..., 125, 128, 125, 126, 120, 125, 125]
features: ['dimuon_mass', 'dimuon_pt', 'dimuon_pt_log', 'dimuon_eta', 'dimuon_cos_theta_cs', 'dimuon_phi_cs', 'jet1_pt_nominal', 'jet1_eta_nominal', 'jet1_phi_nominal', 'jet1_qgl_nominal', 'jet2_pt_nominal', 'jet2_eta_nominal', 'jet2_phi_nominal', 'jet2_qgl_nominal', 'jj_mass_nominal', 'jj_mass_log_nominal', 'jj_dEta_nominal', 'rpt_nominal', 'll_zstar_log_nominal', 'mmj_min_dEta_nominal', 'nsoftjets5_nominal', 'htsoft2_nominal']
df:        dimuon_mass   dimuon_pt  dimuon_pt_log  dimuon_eta  \
entry                                                       
0       126.306807   64.834205       4.171833    0.265019   
1       124.914562  181.915749       5.203544    1.286440   
2       121.604270   65.237430       4.178033   -1.824468   
3       124.125347   51.643715       3.944368   -1.838996   
4       125.454525  192.410311       5.259630    0.339478   

       dimuon_cos_theta_cs  dimuon

In [4]:
filenames

NameError: name 'filenames' is not defined